In [ ]:
import pandas as pd
import numpy as np
import datetime
import networkx as nx
from tqdm import tqdm

First of all we decompressed the '.gz' files and then added a row as a header ('from to time').

In [ ]:
#load data
answers_to_questions = pd.read_table('answers_to_questions', sep=' ')
comments_to_answers = pd.read_table('comments_to_answers', sep=' ')
comments_to_questions = pd.read_table('comments_to_questions', sep=' ')

Let's just take the data from the first half of 2009.

In [ ]:
#create timestamps for the first day of 2009 and the last day of June 2009
import time
start = '01/01/2009'
end = '30/06/2009'
start = int(time.mktime(datetime.datetime.strptime(start, "%d/%m/%Y").timetuple()))
end = int(time.mktime(datetime.datetime.strptime(end, "%d/%m/%Y").timetuple()))

In [ ]:
#take only data from the first half of 2009
answers_to_questions = answers_to_questions[(answers_to_questions.time >= start) & (answers_to_questions.time <= end)]
comments_to_answers = comments_to_answers[(comments_to_answers.time >= start) & (comments_to_answers.time <= end)]
comments_to_questions = comments_to_questions[(comments_to_questions.time >= start) & (comments_to_questions.time <= end)]
#reset the index
answers_to_questions.reset_index(drop = True, inplace = True)
comments_to_answers.reset_index(drop = True, inplace = True)
comments_to_questions.reset_index(drop = True, inplace = True)

In [ ]:
#convert timestamps
for i in tqdm(range(len(answers_to_questions))):
    answers_to_questions.time[i] = datetime.datetime.fromtimestamp(answers_to_questions.time[i]).strftime('%d, %m, %Y')
    
for i in tqdm(range(len(comments_to_answers))):
    comments_to_answers.time[i] = datetime.datetime.fromtimestamp(comments_to_answers.time[i]).strftime('%d, %m, %Y')
    
for i in tqdm(range(len(comments_to_questions))):
    comments_to_questions.time[i] = datetime.datetime.fromtimestamp(comments_to_questions.time[i]).strftime('%d, %m, %Y')

Let's create csv files for each graph in order to never run the previous code again.

In [ ]:
answers_to_questions.to_csv('answers_to_questions.csv', index = False)
comments_to_answers.to_csv('comments_to_answers.csv', index = False)
comments_to_questions.to_csv('comments_to_questions.csv', index = False)

Read the data!

In [ ]:
answers_to_questions = pd.read_csv('answers_to_questions.csv')
comments_to_answers = pd.read_csv('comments_to_answers.csv')
comments_to_questions = pd.read_csv('comments_to_questions.csv')

Each dictionary represents a adjacency list. 

The dictionary *Graph* represent the merge of the 3 single graphs: for each node *u* there are all the nodes *v* , represented as dictionaries, which answered to a question of node *u*, commented an answer of node *u* or commented a question of node *u*. This graph is directed and self-connections are allowed. Each dictionary represented by the nodes *v* cointains a list of all the dates of the actions.

We can think at the dictionary *Graph* as am adjacency list. Moreover each link is weigthed, and the weight is equal to the number of actions taken, which can be calculated by taking the length of the list inside each dictionary *v*.

In [ ]:
answers_to_questions_graph = {}

for i in tqdm(range(len(answers_to_questions))):
    if answers_to_questions['from'][i] not in answers_to_questions_graph:
        answers_to_questions_graph[answers_to_questions['from'][i]] = []
    answers_to_questions_graph[answers_to_questions['from'][i]].append((answers_to_questions.to[i], answers_to_questions.time[i]))
    
    
supp = {}
for key in answers_to_questions_graph.keys():
    for i in range(len(answers_to_questions_graph[key])):
        if answers_to_questions_graph[key][i][0] not in supp:
            supp[answers_to_questions_graph[key][i][0]] = []
        supp[answers_to_questions_graph[key][i][0]].append(answers_to_questions_graph[key][i][1])
    answers_to_questions_graph[key] = supp

In [ ]:
comments_to_answers_graph = {}

for i in tqdm(range(len(comments_to_answers))):
    if comments_to_answers['from'][i] not in comments_to_answers_graph:
        comments_to_answers_graph[comments_to_answers['from'][i]] = []
    comments_to_answers_graph[comments_to_answers['from'][i]].append((comments_to_answers.to[i], comments_to_answers.time[i]))
    

supp = {}
for key in comments_to_answers_graph.keys():
    for i in range(len(comments_to_answers_graph[key])):
        if comments_to_answers_graph[key][i][0] not in supp:
            supp[comments_to_answers_graph[key][i][0]] = []
        supp[comments_to_answers_graph[key][i][0]].append(comments_to_answers_graph[key][i][1])
    comments_to_answers_graph[key] = supp

In [ ]:
comments_to_questions_graph = {}

for i in tqdm(range(len(comments_to_questions))):
    if comments_to_questions['from'][i] not in comments_to_questions_graph:
        comments_to_questions_graph[comments_to_questions['from'][i]] = []
    comments_to_questions_graph[comments_to_questions['from'][i]].append((comments_to_questions.to[i], comments_to_questions.time[i]))
    

supp = {}
for key in comments_to_questions_graph.keys():
    for i in range(len(comments_to_questions_graph[key])):
        if comments_to_questions_graph[key][i][0] not in supp:
            supp[comments_to_questions_graph[key][i][0]] = []
        supp[comments_to_questions_graph[key][i][0]].append(comments_to_questions_graph[key][i][1])
    comments_to_questions_graph[key] = supp

In [ ]:
graph = {}

for i in tqdm(range(len(answers_to_questions))):
    if answers_to_questions['from'][i] not in graph:
        graph[answers_to_questions['from'][i]] = []
    graph[answers_to_questions['from'][i]].append((answers_to_questions.to[i], answers_to_questions.time[i]))
    
for i in tqdm(range(len(comments_to_answers))):
    if comments_to_answers['from'][i] not in graph:
        graph[comments_to_answers['from'][i]] = []
    graph[comments_to_answers['from'][i]].append((comments_to_answers.to[i], comments_to_answers.time[i]))
    
for i in tqdm(range(len(comments_to_questions))):
    if comments_to_questions['from'][i] not in graph:
        graph[comments_to_questions['from'][i]] = []
    graph[comments_to_questions['from'][i]].append((comments_to_questions.to[i], comments_to_questions.time[i]))
    
    
    
supp = {}
for key in graph.keys():
    for i in range(len(graph[key])):
        if graph[key][i][0] not in supp:
            supp[graph[key][i][0]] = []
        supp[graph[key][i][0]].append(graph[key][i][1])
    graph[key] = supp

In [ ]:
comments_to_answers_graph[3381]